In [1]:
import os
import json
import math

import numpy as np

import optuna

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from xgboost import XGBRegressor

In [2]:
random_state = 42

In [3]:
assert os.path.exists('hw_final_open_data.npy'), 'Please, download `hw_final_open_data.npy` and place it in the working directory'
assert os.path.exists('hw_final_open_target.npy'), 'Please, download `hw_final_open_target.npy` and place it in the working directory'
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

In [4]:
def plot_feature_distributions(data, target):
    num_features = data.shape[1]
    num_rows = int(np.ceil((num_features + 1) / 2))
    num_cols = 2
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 3 * num_rows))
    for i in range(num_features):
        row_idx = i // num_cols
        col_idx = i % num_cols
        sns.kdeplot(data[:, i], fill=True, ax=axes[row_idx, col_idx])
        axes[row_idx, col_idx].set_title(f'Feature {i}')
        axes[row_idx, col_idx].set_xlabel('Value')
        axes[row_idx, col_idx].set_ylabel('Density')
    row_idx = num_features // num_cols
    col_idx = num_features % num_cols
    sns.kdeplot(target, fill=True, ax=axes[row_idx, col_idx])
    axes[row_idx, col_idx].set_title('Target')
    axes[row_idx, col_idx].set_xlabel('Value')
    axes[row_idx, col_idx].set_ylabel('Density')
    for i in range(num_features + 1, num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])
    plt.tight_layout()
    plt.show()

def plot_correlation_matrix(data, target):
    combined_data = np.column_stack((data, target))
    correlation_matrix = np.corrcoef(combined_data, rowvar=False)
    plt.figure(figsize=(13, 5))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.6f', 
                xticklabels=[f'Feature {i}' for i in range(data.shape[1])] + ['Target'],
                yticklabels=[f'Feature {i}' for i in range(data.shape[1])] + ['Target'])
    plt.title('Correlation Matrix')
    plt.show()

In [5]:
def my_transformation(data: np.ndarray):
    data = (data - np.array([-5.62410914e-01,  2.58281096e-01, -5.68630481e-01, -6.69724427e-04,
       -5.00564651e-01,  1.51947504e+00, -5.75437567e-01])) * np.array([5.16195921e+02, 1.94217464e+00, 3.64271476e+08, 3.88311869e+00,
       6.18311049e+01, 8.10959916e-01, 2.74664694e+08])
    
    # data = np.c_[data, np.exp(data[:, 5] + data[:, 0])]
    return data

In [6]:
data = my_transformation(data)

In [7]:
# feature selection
feature_importance_model = RandomForestRegressor(random_state=random_state)

feature_importance_model.fit(data, target)
feature_importances = feature_importance_model.feature_importances_
print('Feature importances:\t\t', ',  '.join([f'{index}: {importance:.6f}' for index, importance in list(enumerate(feature_importances))]))
sorted_features = sorted(list(enumerate(feature_importances)), key=lambda x: x[1], reverse=True)
print('Sorted feature importances:\t', ',  '.join([f'{index}: {importance:.6f}' for index, importance in sorted_features]))
sfm = SelectFromModel(feature_importance_model, threshold=0.0003)
data = sfm.fit_transform(data, target)
selected_features = np.where(sfm.get_support())[0]
print("Selected features:\t\t", ', '.join(map(str, selected_features)))

Feature importances:		 0: 0.221282,  1: 0.220723,  2: 0.000235,  3: 0.152597,  4: 0.177167,  5: 0.227647,  6: 0.000348
Sorted feature importances:	 5: 0.227647,  0: 0.221282,  1: 0.220723,  4: 0.177167,  3: 0.152597,  6: 0.000348,  2: 0.000235
Selected features:		 0, 1, 3, 4, 5, 6


In [8]:
# model tuning
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.25, random_state=random_state)
def objective(trial):
    alpha = trial.suggest_float('alpha', 0.01, 2.0, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
    
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=5000, random_state=random_state)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    return mse
    
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

In [9]:
# model
model = ElasticNet(**study.best_params, max_iter=5000, random_state=random_state)

In [10]:
# cross validation
k_fold = KFold(n_splits=4, shuffle=True, random_state=random_state)
cv_scores = -cross_val_score(model, data, target, cv=k_fold, scoring='neg_mean_squared_error', n_jobs=-1)
mse = cv_scores.mean()
print(f"Mean MSE: {mse:.4f}")
print(f"Points: {(1 - mse) * 1.5:.4f}")
print(f"STD MSE: {cv_scores.std():.4f}")
baseline_mse = 0.0905
print(f'Baseline MSE: {baseline_mse}')
print(f'Baseline points: {(1 - baseline_mse) * 1.5:.4f}')

Mean MSE: 0.0906
Points: 1.3641
STD MSE: 0.0018
Baseline MSE: 0.0905
Baseline points: 1.3642


In [11]:
# model fitting
model.fit(data, target)

ElasticNet(alpha=0.010007377571100706, l1_ratio=0.5718048100298648,
           max_iter=5000, random_state=42)

In [12]:
original_predictions = model.predict(data)
rounded_predictions = data.dot(np.round(model.coef_.squeeze(), 4)) + np.round(model.intercept_.squeeze(), 4)

assert np.allclose(original_predictions, rounded_predictions, atol=1e-3)

In [13]:
w_list = list(np.round(model.coef_.squeeze(), 4))
print(f'w_submission = {list(np.round(model.coef_.squeeze(), 4))}\nb_submission = {np.round(model.intercept_.squeeze(), 4)}')

w_submission = [0.0, 0.7972, 0.5483, 0.0723, 0.7532, 0.0]
b_submission = 3.7086


In [14]:
assert len(w_list) + 1 <= 15